### Running gather
Want to compare:
human: 2850
pig: 3217
- GTDB reps
- GTDB all
- fungi
- mags (alone, drep) (done/done)
- MAGs all + GTDB all (done/done)
- MAGs99 + GTDB reps 
- animal + plant (done/done)
- protozoa


In [ ]:
# accidentally did part of gtdb_genomes. moved results to: 
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sourmash/check/gtdb_genomes/
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sourmash/gather_huma/gtdb_genomes
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sourmash/gather/gtdb_genomes


In [ ]:
# merge signatures. 
srun --account=ctbrowngrp -p med2 -J calc-pres -t 4:00:00 -c 24 --mem=80gb --pty bash

# run the script
mamba activate branchwater-skipmer

sourmash sig merge MAGs.all.zip -k 21 -o MAGs.all.merged.zip

In [ ]:
# calculate hash presence for all 3217 metagenomes
srun --account=ctbrowngrp -p bmm -J calc-pres -t 8:00:00 -c 120 --mem=120gb --pty bash

# run the script
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 120 --rerun-incomplete -k -s gather.smk -n

snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 120 --rerun-incomplete -k -s gather_human.smk -n

In [ ]:
# calculate hash presence for all 3217 metagenomes
srun --account=ctbrowngrp -p med2 -J calc-pres -t 4:00:00 -c 120 --mem=100gb --pty bash

# run the script
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=100000 --rerun-triggers mtime \
-c 120 --rerun-incomplete -k -s gather_human.smk -n

snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 120 --rerun-incomplete -k -s gather_pangenomedb.smk -n

## Gather against GTDB
- It seems that gtdb all genomes performs pretty well. 
- Make pangenomedb from GTDB genomes and run against human and pig samples
- Also subtract the hashes in all pig gut from the GTDB. --> How much is already contained in there?
- Do gather against protozoa and fungi. 

In [ ]:
ANIMAL_PLANT_DB_K51 = '/group/ctbrowngrp5/sourmash-db/entire-2025-01-21/animals_plants-2025-01-21.k51.rocksdb'
ALL_PANG = '/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all.pangenomedb.k21.zip'
ALL_MAG = '/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all.zip'
MAG95 = '/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.95.pangenomedb.zip'
MAG99 = '/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.99.pangenomedb.zip'
GTDB = '/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.k21.rocksdb'
GTDB_REPS = '/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226-reps.k21.rocksdb'
FUNGI = '/group/ctbrowngrp5/sourmash-db/genbank-euks-2024.01/fungi.k21.sig.zip'

## Gather will also need to be done for the human samples
- On GTDB only (done)
- On MAGs only
- On MAGs + GTDB

In [ ]:
# calculate hash presence for all 2849 human metagenomes
srun --account=ctbrowngrp -p med2 -J calc-pres -t 6:00:00 -c 32 --mem=100gb --pty bash
# run the script
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 60 --rerun-incomplete -k -s gather_human.smk -n

snakemake --use-conda --resources mem_mb=100000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -s gather_human.smk -n

mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=100000 --rerun-triggers mtime \
-c 32 --rerun-incomplete -k -s gather.smk -n

In [ ]:
sourmash gather /group/ctbrowngrp/irber/data/wort-data/wort-sra/sigs/SRR27029883.sig /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/compare_gtdb_mags/gtdb-rs226.pangenomedb.10k.zip --create-empty-results        -k 21 --scaled 10000 -o ../results/sourmash_human/gather/SRR27029883.gtdb.k21.csv && touch ../results/sourmash_human/check/SRR27029883.gtdb.k21.check

In [ ]:
# calculate hash presence for all 2849 human metagenomes
srun --account=ctbrowngrp -p med2 -J calc-pres -t 4:00:00 -c 100 --mem=120gb --pty bash
srun --account=ctbrowngrp -p med2 -J calc-pres -t 2:00:00 -c 8 --mem=80gb --pty bash


# run the script
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 100 --rerun-incomplete -k -s gather_human.smk -n

mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 8 --rerun-incomplete -k -s gather.smk -n

In [ ]:
# Using a script to sum all f_unique_qeighted for all these
python count_funique_weighted.py

## Concatenate the gather output
- Count what species are found the most times (presence absence)
- Subset the human results to only usable metaGs. 


In [ ]:
# move human metaG results for good metags

while read basename; do
  mv /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sourmash/gather_human/"$basename"* /group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sourmash/gather_human_good/
done < metag-wort-human-good.txt


In [ ]:
# count the gather sums for the gtdb outputs 
# need species counts for humans and pigs
srun --account=ctbrowngrp -p med2 -J calc-gather -t 1:00:00 -c 2 --mem=100gb --pty bash

mamba activate pangenomics_dev


sourmash scripts gather_tables \
gather/*.gtdb.csv \
-c "intersect_bp" -p \
-o 250813_gatherout.gtdb.species.pig.csv 

sourmash scripts gather_tables \
gather/*.mags_and_gtdb.csv \
-c "intersect_bp" -p \
-o 250813_gatherout.gtdbmags.species.pig.csv 

sourmash scripts gather_tables \
gather/*.gtdb.csv \
-c "f_unique_weighted" \
-o 250813_gatherout.gtdb.species.pig_funique.csv 

sourmash scripts gather_tables \
gather/*.mags_and_gtdb.csv \
-c "f_unique_weighted" -l species -t ../ \
-o 250813_gatherout.gtdbmags.species.pig_funique.csv 








In [ ]:
# Maybe make a table at each phylogenetic level. 

In [ ]:
# need some sort of tax file
csvtk concat /group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226-reps.lineages.csv \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/MAGs/250528_mag_tax.csv > 250602_taxgtdb.pig.csv

sed 's/$/,/' 250602_taxgtdb.pig.csv > 250602_taxgtdb.st.pig.csv

In [ ]:
# could i do fastmultigather?
# index as rocksdb?

mamba activate branchwater-skipmer
sourmash index gtdb-rs226-reps.pangenomedb.rocksdb -F rocksdb -k 21 gtdb-rs226-reps.pangenomedb.10k.zip


srun --account=ctbrowngrp -p med2 -J calc-pres -t 1:00:00 -c 1 --mem=50gb --pty bash


In [ ]:
srun --account=ctbrowngrp -p med2 -J fmg -t 1:00:00 -c 100 --mem=100gb --pty bash

# do fmg against a concatenated database
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/3217_metag.txt \
MAGs.all_k31.merged.zip -k 31 -s 10000 -c 100 -o 250808_fmg_MAGs_merged.csv



In [ ]:
# do fmg against the non-concatenated databases, for 50 metagenomes first
# Maybe even 10? The results just need to be the same as from merged
srun --account=ctbrowngrp -p med2 -J fmg -t 02:00:00 -c 100 --mem=80gb --pty bash
mamba activate branchwater-skipmer

# gtdb reps
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/100_metag.txt \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226-reps.k21.rocksdb \
 -k 21 -s 10000 -c 100 -t 0 -o 250731_fmg_gtdb-reps_t0.csv -q

# gtdb genomes
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/100_metag.txt \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226.k21.rocksdb \
 -k 21 -s 10000 -c 100 -t 0 -o 250731_fmg_gtdb-genomes_t0.csv -q

# MAGs
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/100_metag.txt \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all.rocksdb \
 -k 21 -s 10000 -c 100 -t 0 -o 250731_fmg_MAGs_t0.csv -q

In [ ]:
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all.merged.rocksdb
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/3217_metag.txt \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all.merged.rocksdb \
 -k 21 -s 10000 -c 100  -o 250731_fmg_MAGs.merged.csv -q


sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/3217_metag.txt \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all.merged.rocksdb \
 -k 21 -s 10000 -c 100 -t 0 -o 250731_fmg_MAGs.merged.t0.csv -q

In [ ]:
# gotta redo the gather for mags-nonmerged bc it gives merged results
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/3217_metag.txt \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/sketches/MAGs.all.rocksdb \
 -k 21 -s 10000 -c 100 -o 250801_fmg_MAGs.nonmerged.csv -q

 # same for 99 and 95
sourmash index MAGs.99.rocksdb -F rocksdb -k 21 MAGs.99.zip
sourmash index MAGs.95.rocksdb -F rocksdb -k 21 MAGs.95.zip

